In [13]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 128
tl.set_backend('pytorch')
help(tl.set_backend)

Help on function set_backend in module tensorly:

set_backend(backend_name)
    Sets the backend for TensorLy
    
        The backend will be set as specified and operations will used that backend
    
    Parameters
    ----------
    backend_name : {'mxnet', 'numpy', 'pytorch', 'tensorflow', 'cupy'}, default is 'numpy'



Using pytorch backend.


In [14]:
tag = [72, 71, 84, 54, 57, 65, 64]
Y = np.zeros(467)
print(Y.shape)
Y[0 : tag[0]] = 0
Y[tag[0] : tag[0]+tag[1]] = 1
Y[tag[0]+tag[1] : tag[0]+tag[1]+tag[2]] = 2
Y[tag[0]+tag[1]+tag[2] : tag[0]+tag[1]+tag[2]+tag[3]] = 3
Y[tag[0]+tag[1]+tag[2]+tag[3] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]] = 4
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]] = 5
Y[tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5] : tag[0]+tag[1]+tag[2]+tag[3]+tag[4]+tag[5]+tag[6]] = 6
Y = Y.tolist()

(467,)


In [15]:
inputs = []
MAX_LENGTH = 53
for i in range(0,467):
    a = np.loadtxt('/Users/denhiroshi/Downloads/Embeding/ans_1.0_'+str(i)+'.csv',dtype=np.float64,delimiter=',')
    a = a.transpose(1,0)
    a = np.exp(a)
    a = np.sum(a,axis=1)
    a = np.log(a)
    if a.shape[0] < MAX_LENGTH:
        add = MAX_LENGTH - a.shape[0]
        adds = np.zeros(add).astype(np.float64)
        a = np.concatenate((a, adds))
    inputs.append(a)
inputs[0].shape

(53,)

In [16]:
use = np.stack(inputs)
mean = use.mean()         #计算平均数
deviation = use.std()     #计算标准差
# 标准化数据的公式: (数据值 - 平均数) / 标准差
use = (use - mean) / deviation
use.shape

(467, 53)

In [17]:
b = np.fromfile('glove_WV.dat',dtype=np.float64).reshape((467,-1,200))
position_tag = np.loadtxt('position_tag.csv',dtype=np.float64,delimiter=',')

In [18]:
temp = [i for i in range(53)]
c = []
for i in range(467):
    position_a = (np.array(temp) - position_tag[i][0]).reshape((-1,1))
    position_b = (np.array(temp) - position_tag[i][1]).reshape((-1,1))
    position = np.concatenate((position_a,position_b),axis = 1)
    c.append(np.concatenate((b[i],position),axis = 1))
c = np.stack(c)
b = c
b.shape

(467, 53, 202)

In [19]:
inputs = []
for i in range(467):
    inputs.append((torch.from_numpy(b[i]),torch.from_numpy(use[i]),Y[i],position_tag[i].astype(np.int64)))
shuffle(inputs)
flag = int(len(inputs)*0.7)
train = inputs[:flag]
test = inputs[flag:]

In [20]:
t_s = []
t_e = []
t_y = []
t_p = []

for E,S,Y,pos in test:
    t_s.append(S)
    t_e.append(E)
    t_y.append(Y)
    t_p.append(pos)
t_s = torch.stack(t_s)
t_e = torch.stack(t_e)
t_p = torch.from_numpy(np.stack(t_p))
t_y = torch.from_numpy(np.array(t_y))
t_s.shape, t_e.shape, t_y.shape, t_p.shape

(torch.Size([141, 53]),
 torch.Size([141, 53, 202]),
 torch.Size([141]),
 torch.Size([141, 2]))

# 0.8226950354609929 18轮

In [31]:
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, 53, 200)
            nn.Conv2d(
                in_channels=1,      # input height
                out_channels=53,    # n_filters
                kernel_size=(1,203),      # filter size
            ),      # output shape (16, 53, 200)
            nn.ReLU(),    # activation
        )
        self.maxpool = nn.MaxPool2d(kernel_size=(53,1))   # 在 2x2 空间里向下采样, output shape (16, 53, 1)
        self.output = nn.Linear(159, 7)   # fully connected layer, output 10 classes

    def forward(self, sen, x, pos):
        torch.set_default_dtype(torch.double)
        sen = sen.reshape((sen.shape[0],1,sen.shape[1])) # (BATCH_SIZE,1,53)
        use = torch.bmm(sen.double(),x) # (BATCH_SIZE,1,202)
        use = torch.sigmoid(use) # (BATCH_SIZE,1,202)
        use = torch.softmax(use,2) # (BATCH_SIZE,1,202)
        use = use.reshape((use.shape[0],-1,1))
        use = torch.bmm(x,use)
        x = torch.cat((x,use),2)
        
        x = x.reshape((x.shape[0],1,x.shape[1],-1)).double()
        x = self.conv1(x.double())
        
        x1 = torch.zeros(x.shape)
        x2 = torch.zeros(x.shape)
        x3 = torch.zeros(x.shape)
        
        min_x1 = torch.ones(x.shape) * (-100)
        min_x2 = torch.ones(x.shape) * (-100)
        min_x3 = torch.ones(x.shape) * (-100)
        
        for i in range(pos.shape[0]):
            x1[i, :, :pos[i][0], :]
            min_x1[i, :, :pos[i][0], :]  = 0.0
            x2[i, :, pos[i][0]:pos[i][1], :]  = 1.0
            min_x2[i, :, pos[i][0]:pos[i][1], :] = 0.0
            x3[i, :, pos[i][1]:, :] = 1.0
            min_x3[i, :, pos[i][1]:, :] = 0.0
            
         # 卷积结果*mask得到分段后的卷积向量
        x1 *= x
        x2 *= x
        x3 *= x
        
        # 将无用部分赋予最小值，避免影响maxpool操作
        x1 += min_x1
        x2 += min_x2
        x3 += min_x3
        
        # 分段池化
        x1 = self.maxpool(x1)
        x2 = self.maxpool(x2)
        x3 = self.maxpool(x3)
        
        # 展平
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        x3 = x3.view(x3.size(0), -1)
        
        
        x = torch.cat((x1,x2,x3),dim=1)

        output = self.output(x)
        return output

In [ ]:
# 创建数据加载器
loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,      # mini batch size
    shuffle = True,               # 设置随机洗牌
    num_workers = 2,              # 加载数据的进程个数
)
net = CNN()
optimizer = torch.optim.Adam(net.parameters(),lr=0.01)
loss_F = F.cross_entropy
print(net)
start = time.time()
for epoch in range(1,201): 
    for step, (batch_x, sen, batch_y,pos) in enumerate(loader):
        pred = net(sen,batch_x,pos)
        loss = loss_F(pred,batch_y.long()) # 计算loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 1 == 0 and step == 2: # 每50步，计算精度
            with torch.no_grad():
                test_pred = net(t_s,t_e,t_p)
                prob = torch.nn.functional.softmax(test_pred,dim=1)
                pred_cls = torch.argmax(prob,dim=1)
                acc = (pred_cls.int() == t_y.int()).sum().numpy() / pred_cls.size()[0]
                print(f"{epoch}: accuracy:{acc} loss: {loss} time: {time.time() - start}")

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 53, kernel_size=(1, 203), stride=(1, 1))
    (1): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=(53, 1), stride=(53, 1), padding=0, dilation=1, ceil_mode=False)
  (output): Linear(in_features=159, out_features=7, bias=True)
)
1: accuracy:0.2695035460992908 loss: 6.38185374282745 time: 0.31663012504577637
2: accuracy:0.5106382978723404 loss: 5.435771316710462 time: 0.6906559467315674
3: accuracy:0.6524822695035462 loss: 4.660417435807111 time: 1.0702130794525146
4: accuracy:0.6312056737588653 loss: 3.441575977732122 time: 1.3829050064086914
5: accuracy:0.7446808510638298 loss: 1.6585040217357736 time: 1.7395830154418945
6: accuracy:0.6950354609929078 loss: 1.606130030754666 time: 2.053232192993164
7: accuracy:0.7163120567375887 loss: 0.7283988401613254 time: 2.368453025817871
8: accuracy:0.7304964539007093 loss: 1.7811122893831415 time: 2.690631866455078
9: accuracy:0.7163120567375887 loss: 0.3371134581898572 time: 2.9972379207611084
10: ac